# Read Me File: 

## FinCEN SAR Data Analysis: Consumer Fraud & Identity TheftThis repository contains the analysis of Suspicious Activity Reports SARs data sourced from the Financial Crimes Enforcement Network. The goal is to uncover patterns, trends, and correlations related to consumer fraud and identity theft within the U.S. financial system. 
## The project utilized FinCEN datasets of known SAR to gain insight into the payment methods and instruments utilized when suspicious activity is reported. Federal Data: Available for the period 2014–2024. State Data: Available for the period 2020–2025.The datasets contain crucial information associated with various industry types (e.g., depository institutions, money services businesses, securities/futures).
## Project Objectives: This analysis is structured around two primary objectives: 
## Objective 1: Data Preparation Perform Exploratory Data Analysis EDA, data cleaning, and standardization/normalization of the SAR datasets to ensure accuracy and consistency for analysis. 
## Objective 2: Data Visualization & Insights Explore the cleaned data and generate visualizations to report findings and answer key investigative questions: 
## 1. Temporal Trends: How has the total number of SAR changed over time (monthly, quarterly, or yearly) for each institution type? 
## Identify any seasonal trends or sudden spikes in reporting.
## 2. Payment Method Growth: Which payment methods show the highest growth rate in suspicious activity?
## 3. Payment Method Correlation: What correlations exist between various payment methods and instruments reported in the SARs?
## 4. Geographic Hotspots (Federal): What are the top 5 states with the highest number of?
## 5. Geographic Hotspots (Local): What are the top 5 counties with the highest number of SARs?
## 6. Activity Classification: What are the most common types of suspicious activity reported (e.g., Money Laundering, Structuring, Identity Theft)?## ## 7. Anomalies: Identify any anomalous reporting periods, and determine the activity types most impacted during those times.
## 8 Pandemic Impact: How did the COVID-19 pandemic (2020–2022) affect different types of financial crimes reported via SARs?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import folium 
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster


In [2]:
# Cleaning function and reading in sheet Exhibit 1 
def clean_year_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df[df['Month'].notna()]  # only keeping rows with a value in 'Month'
    df = df[~df[first_col].astype(str).str.contains('Total|Subtotal', case=False, na=False)]
    df = df.reset_index(drop=True)
    df.to_csv('year1_df.csv', index=False)
    return df

file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
year1_df = pd.read_excel(file_path, sheet_name='Exhibit 1', header=13)
clean_year1_df = clean_year_df(year1_df)

In [3]:
import os
print(os.getcwd())

C:\Users\Jen_L


In [4]:
year1_df.head()

,Month,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,January,65898.0,66101.0,70460.0,75234.0,75619.0,88130,100171.0,97471.0,131949.0,158497.0,165663.0
1,February,61637.0,65984.0,73927.0,73806.0,71908.0,86497,90721.0,98318.0,134587.0,157286.0,165823.0
2,March,64462.0,73420.0,83964.0,88071.0,87326.0,97933,101588.0,116790.0,165396.0,191348.0,165540.0
3,April,73302.0,74049.0,81282.0,77383.0,78763.0,91088,102478.0,114397.0,157832.0,161100.0,168409.0
4,May,75301.0,68216.0,80822.0,77500.0,84790.0,95166,88941.0,109821.0,149970.0,171810.0,174560.0


In [5]:
# Cleaning function and reading in sheet Exhibit 2
def clean_year_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    # Removing rows where the first column contains summary/total labels
    df = df[~df[first_col].astype(str).str.contains('Total|Subtotal', case=False, na=False)]
    df = df.reset_index(drop=True)
    df.to_csv('year2_df.csv', index=False)
    return df

file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
year2_df = pd.read_excel(file_path, sheet_name='Exhibit 2', header=13)
clean_year2_df = clean_year_df(year2_df)


In [6]:
clean_year2_df.head()

,State/Territory,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Total
0,Alabama,8932,10961,13022,13979,15742,16695,20606,29274,31073,28313,32660,221257
1,Alaska,1892,2280,2683,2610,2577,2226,2377,2484,2978,3469,3749,29325
2,American Samoa,553,110,79,71,40,90,37,19,4,3,7,1013
3,Arizona,18023,16733,18829,18987,17692,16501,20467,23776,26196,28124,28029,233357
4,Arkansas,3329,4381,4771,5800,5347,6890,8420,10113,9792,10751,11916,81510


In [7]:
# Cleaning function and reading in sheet Exhibit 3
def clean_rank_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]    
    df = df.reset_index(drop=True)
    df.to_csv('rank1_df.csv', index=False)
    return df
    
file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
rank1_df = pd.read_excel(file_path, sheet_name='Exhibit 3', header=13)
clean_rank1_df = clean_rank_df(rank1_df)


In [8]:
clean_rank1_df.head()

,Rank,State/Territory,Filings (Overall),Percentage (Overall),Rank.1,State/Territory.1,Filings (Overall).1,Percentage (Overall).1
0,1,California,2291535,0.129301,31,Oklahoma,132202,Less than 1%
1,2,Ohio,1445487,0.081562,32,Mississippi,130258,Less than 1%
2,3,New York,1423928,0.080346,33,Oregon,124186,Less than 1%
3,4,North Carolina,1252861,0.070693,34,Puerto Rico,113601,Less than 1%
4,5,Texas,1196373,0.067506,35,Kentucky,94442,Less than 1%


In [9]:
# Cleaning function and reading in sheet Exhibit 4
def clean_rank_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]   
    df = df.reset_index(drop=True)
    df.to_csv('rank2_df.csv', index=False)
    return df
    
file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
rank2_df = pd.read_excel(file_path, sheet_name='Exhibit 4', header=13) 
clean_rank2_df = clean_rank_df(rank2_df)



In [10]:
clean_rank2_df.head()

,Rank,Suspicious Activity Type,Filings (Overall),Percentage (Overall)
0,1,Suspicion concerning the source of funds,3538283,0.088073
1,2,Transaction(s) below CTR threshold,3515701,0.08751
2,3,Check,2919906,0.07268
3,4,"Transaction with no apparent economic, busines...",2917251,0.072614
4,5,Transaction out of pattern for customer(s),2498336,0.062187


In [11]:
# Cleaning function and reading in sheet Exhibit 5
def clean_year_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df.reset_index(drop=True)
    df.to_csv('year5_df.csv', index=False)
    return df
    
file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
year5_df = pd.read_excel(file_path, sheet_name='Exhibit 5', header=13)
clean_year5_df = clean_year_df(year5_df)


In [12]:
clean_year5_df.head()

,Suspicious Activity Category,Suspicious Activity Type,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Cyber Eventàà,Against financial institution(s)*,-,-,-,-,260,2829,4871,5399,7188,7161,5891
1,Fraud,ACH,24904,37338,48679,50680,65179,88858,143269,176911,183730,199939,204927
2,Gaming Activitiesà,Chip walking*,-,-,-,-,4,9,7,2,15,7,11
3,Identification Documentation,Changes spelling or arrangement of name,1304,1103,924,884,1002,893,682,728,752,782,774
4,Insurance,Excessive insurance,25,23,19,21,25,25,32,29,26,28,53


In [13]:
# Cleaning function and reading in sheet Exhibit 6
def clean_year_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df.reset_index(drop=True)
    df.to_csv('year6_df.csv', index=False)
    return df
    
file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
year6_df = pd.read_excel(file_path, sheet_name='Exhibit 6', header=13)
clean_year6_df = clean_year_df(year6_df)



In [14]:
clean_year6_df.head()

,Regulator,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Commodity Futures Trading Commission,0,0,2,0,0,0,1,0,0,0,0
1,Comptroller of the Currency,586728,592490,627894,584581,622773,716376,737470,845337,1170818,1258034,1261623
2,Federal Deposit Insurance Corporation,102803,116655,142419,120567,126197,141862,169918,201968,232510,231606,244350
3,Federal Housing Finance Agency,0,0,0,0,0,91,0,0,3,0,0
4,Federal Reserve Board,76318,82914,91901,101656,108067,123208,122553,157865,184089,182076,236444


In [15]:
# Cleaning function and reading in sheet Exhibit 7
def clean_year_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df.reset_index(drop=True)
    df.to_csv('year7_df.csv', index=False)
    return df
    
file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
year7_df = pd.read_excel(file_path, sheet_name='Exhibit 7', header=13)
clean_year7_df = clean_year_df(year7_df)



In [16]:
clean_year7_df.head()

,Relationship,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Accountant,54,74,43,58,51,58,85,86,59,95,52
1,Agent,406,357,402,457,531,479,573,595,647,552,745
2,Appraiser,732,254,199,57,46,44,36,32,31,27,30
3,Attorney,177,106,74,59,48,59,46,48,58,58,65
4,Borrower,49716,42291,42887,43392,49748,52343,50260,52276,51056,61229,62962


In [17]:
# Cleaning function and reading in sheet Exhibit 8
def clean_year_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df.reset_index(drop=True)
    df.to_csv('year8_df.csv', index=False)
    return df
    
file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
year8_df = pd.read_excel(file_path, sheet_name='Exhibit 8', header=13)
clean_year8_df = clean_year_df(year8_df)



In [18]:
clean_year8_df.head()

,Product Type,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Bonds/Notes,227,240,277,283,217,269,265,288,431,486,443
1,Commercial mortgage,456,436,408,435,467,580,438,449,508,562,636
2,Commercial paper,636,715,701,270,219,300,387,420,383,322,345
3,Credit card,85887,98698,104328,119122,162435,172399,124131,109972,224083,249939,238469
4,Debit card,53179,84146,116754,126766,146218,190298,264503,369294,471557,524182,598186


In [19]:
# Cleaning function and reading in sheet Exhibit 9
def clean_year_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df.reset_index(drop=True)
    df.to_csv('year9_df.csv', index=False)
    return df

file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
year9_df = pd.read_excel(file_path, sheet_name='Exhibit 9', header=13)
clean_year9_df = clean_year_df(year9_df)

In [20]:
clean_year9_df.head()

,Type of Instrument Type(s)/\nPayment Mechanism(s),2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Bank/Cashier's check,50706,57748,65792,65582,68622,75471,78299,83715,97359,107850,108479
1,Foreign currency,1988,2014,2366,2251,2062,2088,2480,3942,3953,3603,4112
2,Funds transfer,134705,172940,214503,233941,258424,285219,395467,526198,566531,652865,769551
3,Gaming instruments,176,297,253,243,218,237,319,538,824,644,760
4,Government payment,8293,7983,6956,4749,4434,4205,64574,121087,50298,41772,52035


## Reading and Cleaning state data 2020-2025 

In [21]:
# Identifying the file path for the state/county data for years: 2020-2025
file_paths = [
    r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\2020 County Depository Institutions.csv",
    r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\2021 County Depository Institutions.csv",
    r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\2022 County Depository Institutions.csv",
    r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\2023 County Depository Institutions.csv",
    r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\2024 County Depository Institutions.csv",
    r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\2025 County Depository Institutions.csv"
]

# Creating a function to clean dataframe and to: drop unnamed columns, strip column names, and standardize

def clean_dataframe(df):
    """Drops unnamed columns, strips column names, and drops NaNs."""
    # Dropping empty columns or ones that start with 'Unnamed'
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
    # Stripping whitespaces from column names
    df.columns = df.columns.str.strip() 
    # Dropping rows with any missing values
    df = df.dropna()      
    return df

# Reading each CSV into a dataframe, cleaning it, and storing as a list
all_dataframes = [clean_dataframe(pd.read_csv(f)) for f in file_paths]

# Concatenating all the dataframes into one
state_df = pd.concat(all_dataframes, ignore_index=True)

# Renaming 'Year Month' to 'Year'
state_df = state_df.rename(columns={'Year Month': 'Year'})

def save_cleaned_state_data(df: pd.DataFrame, output_path: Path):
    """Saves the cleaned DataFrame to a CSV file."""
    # Ensure the parent directory exists
    output_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"Cleaned data saved to: {output_path}")

base_dir = Path(r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data")
file_path = base_dir / "Bank_Fraud_Capstone_2025\data" 
output_dir = base_dir / "state_combined_data"

print(state_df.head())

save_cleaned_state_data(state_df, output_dir / "state_combined.csv") 


   Year    State              County                Industry  \
0  2020  Alabama  Autauga County, AL  Depository Institution   
1  2020  Alabama  Autauga County, AL  Depository Institution   
2  2020  Alabama  Autauga County, AL  Depository Institution   
3  2020  Alabama  Autauga County, AL  Depository Institution   
4  2020  Alabama  Autauga County, AL  Depository Institution   

  Suspicious Activity Regulator          Product               Instrument  \
0                 ACH      FDIC  Deposit Account     Bank/Cashier's Check   
1                 ACH      FDIC  Deposit Account  Personal/Business Check   
2                 ACH      FDIC  Deposit Account            U.S. Currency   
3                 ACH      FDIC  Deposit Account                  [Total]   
4                 ACH      FDIC          [Total]                  [Total]   

  Count Countym  
0     1     NaN  
1     1     NaN  
2     1     NaN  
3     3     NaN  
4     3     NaN  
Cleaned data saved to: C:\Users\Jen_L\OneDri

In [22]:
state_df

,Year,State,County,Industry,Suspicious Activity,Regulator,Product,Instrument,Count,Countym
0,2020,Alabama,"Autauga County, AL",Depository Institution,ACH,FDIC,Deposit Account,Bank/Cashier's Check,1,NaN
1,2020,Alabama,"Autauga County, AL",Depository Institution,ACH,FDIC,Deposit Account,Personal/Business Check,1,NaN
2,2020,Alabama,"Autauga County, AL",Depository Institution,ACH,FDIC,Deposit Account,U.S. Currency,1,NaN
3,2020,Alabama,"Autauga County, AL",Depository Institution,ACH,FDIC,Deposit Account,[Total],3,NaN
4,2020,Alabama,"Autauga County, AL",Depository Institution,ACH,FDIC,[Total],[Total],3,NaN
...,...,...,...,...,...,...,...,...,...,...
440579,2025,Wyoming,NaN,Depository Institution,Wire,FDIC,[Total],[Total],1,"Weston County, WY"
440580,2025,Wyoming,NaN,Depository Institution,Wire,[Total],[Total],[Total],1,"Weston County, WY"
440581,2025,Wyoming,NaN,Depository Institution,[Total],[Total],[Total],[Total],4,"Weston County, WY"
440582,2025,Wyoming,NaN,Depository Institution,[Total],[Total],[Total],[Total],162,[Total]


## Concatenating similiarly structured dataframes for Rank 

In [23]:
# Reading in sheets for Exhibit 3 and Exhibit 4

file_path = r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data\Section 2 - Depository Institution SARs.xlsx"
rank1_df = pd.read_excel(file_path, sheet_name='Exhibit 3', header=13)
rank2_df = pd.read_excel(file_path, sheet_name='Exhibit 4', header=13)

In [24]:
# Defining the functions for clean_rank1 and rank 2

def clean_rank1_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df[~df[first_col].astype(str).str.contains('Total|Subtotal', case=False, na=False)]
    df = df.reset_index(drop=True)
    return df

def clean_rank2_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(how='all')
    first_col = df.columns[0]
    df = df[df[first_col].notna()]
    df = df[~df[first_col].astype(str).str.contains('Total|Subtotal', case=False, na=False)]
    df = df.reset_index(drop=True)
    return df

In [25]:
# Creating variables in order to create dataframes for each of the Exhibit sheets 3 and 4 and concatenating the dataframes

clean_rank1_df = clean_rank1_df(rank1_df)
clean_rank2_df = clean_rank2_df(rank2_df)

rankdf_combined = pd.concat(
    [clean_rank1_df, clean_rank2_df],ignore_index=True
)


In [26]:
def save_cleaned_combined_rank_data(df: pd.DataFrame, output_path: Path):
    """Saves the cleaned DataFrame to a CSV file."""
    # Ensure the parent directory exists
    output_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"Cleaned data saved to: {output_path}")

base_dir = Path(r"C:\Users\Jen_L\OneDrive\Documents\NSS\DA15\capstone\Bank_Fraud_Capstone_2025\data")
file_path = base_dir / "Bank_Fraud_Capstone_2025\data" 
output_dir = base_dir / "combined_rank_data"

print(rankdf_combined.head())

save_cleaned_combined_rank_data(rankdf_combined, output_dir / "combined_rank_data.csv") 

   Rank State/Territory  Filings (Overall) Percentage (Overall)  Rank.1  \
0     1      California            2291535             0.129301    31.0   
1     2            Ohio            1445487             0.081562    32.0   
2     3        New York            1423928             0.080346    33.0   
3     4  North Carolina            1252861             0.070693    34.0   
4     5           Texas            1196373             0.067506    35.0   

  State/Territory.1  Filings (Overall).1 Percentage (Overall).1  \
0          Oklahoma             132202.0           Less than 1%   
1       Mississippi             130258.0           Less than 1%   
2            Oregon             124186.0           Less than 1%   
3       Puerto Rico             113601.0           Less than 1%   
4          Kentucky              94442.0           Less than 1%   

  Suspicious Activity Type  
0                      NaN  
1                      NaN  
2                      NaN  
3                      NaN  
4

In [27]:
rankdf_combined['Suspicious Activity Type']

0                                     NaN
1                                     NaN
2                                     NaN
3                                     NaN
4                                     NaN
                      ...                
122                  Unauthorized pooling
123                         Wash trading*
124              Unknown source of chips*
125                         Chip walking*
126    Inquiry about end of business day†
Name: Suspicious Activity Type, Length: 127, dtype: object